In [6]:
import ibis
from ibis import _
import pathlib
from cng.utils import *
duckdb_install_h3()

con = ibis.duckdb.connect(extensions = ["spatial", "h3"])
set_secrets(con)

con.raw_sql("SET threads = 2;")

In [7]:

congress_code = "119"  # 119th Congress
states = con.read_geo("/vsizip//vsicurl/https://www2.census.gov/geo/tiger/TIGER2024/STATE/tl_2024_us_state.zip")
state_fips = states.select("STATEFP").order_by(_.STATEFP).execute()["STATEFP"]

def generate_urls(congress_code, state_fips):
    base_url = f"https://www2.census.gov/geo/tiger/TIGER2024/CD/tl_2024_"
    extension = f"_cd{congress_code}.zip"
    return [base_url + state_code + extension for state_code in state_fips]

## for some reason these won't stream
state_fips = state_fips[state_fips != '45']
state_fips = state_fips[state_fips != '60']

In [8]:
urls = generate_urls(congress_code, state_fips)
for url in urls:
    vurl = "/vsizip//vsicurl/" + url
    dest = "data/" + pathlib.Path(vurl).with_suffix(".parquet").name
    # print(dest)
    con.read_geo(vurl).to_parquet(dest)
    

In [11]:
con.read_geo("/vsizip/tl_2024_45_cd119.zip").to_parquet("data/tl_2024_45_cd119.parquet")
con.read_geo("/vsizip/tl_2024_60_cd119.zip").to_parquet("data/tl_2024_60_cd119.parquet")

In [12]:

df = con.read_parquet("data/**")
df.to_parquet("s3://public-census/year=2024/CD/cd.parquet")



In [26]:

import ibis.expr.datatypes as dt
@ibis.udf.scalar.builtin
def ST_Multi (geom) -> dt.geometry:
    ...
    
def geom_to_cell (df, zoom = 8):
    con = df._find_backend() # df.get_backend() ibis >= 10.0

    # First make sure we are using multipolygons everywhere and not a mix
    cases = ibis.cases(
        (df.geom.geometry_type() == 'POLYGON' , ST_Multi(df.geom)),
        else_=df.geom,
    )
    
    df = df.mutate(geom = cases)
    sql = ibis.to_sql(df)
    expr = f'''
        WITH t1 AS (
        SELECT * EXCLUDE (geom), UNNEST(ST_Dump(ST_GeomFromWKB(geom))).geom AS geom 
        FROM ({sql})
        ) 
        SELECT *, h3_polygon_wkt_to_cells_string(geom, {zoom}) AS h{zoom}  FROM t1
    '''

    out = con.sql(expr)
    return out


In [27]:

df = con.read_parquet("data/**")

geom_to_cell(df, 8).to_parquet("s3://public-census/year=2024/CD/cd-hex-z8.parquet")



FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [30]:
con.read_parquet("s3://public-census/year=2024/CD/cd-hex-z8.parquet").head().mutate(h8 = _.h8.unnest()).to_parquet("test.parquet")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

OutOfMemoryException: Out of Memory Error: failed to allocate data of size 16.0 MiB (25.5 GiB/25.5 GiB used)